<a href="https://colab.research.google.com/github/TorchSpatiotemporal/tsl/blob/main/examples/notebooks/a_gentle_introduction_to_tsl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Gentle Introduction to tsl
---

This a tutorial notebook about __tsl (Torch Spatiotemporal)__, a Python library built upon the PyTorch ecosystem
and tailored for spatiotemporal data processing.

In this notebook we are going to see what are the necessary steps from data loading to model training.

## Installation
---

Let's start by the installation. If you run the notebook in colab, you can install tsl with these commands:

In [ ]:
!git clone https://github.com/TorchSpatiotemporal/tsl.git
!pip install torch==1.10.1+cu111 torchvision==0.11.2+cu111 torchaudio==0.10.1 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torch-scatter torch-sparse torch-geometric -f https://data.pyg.org/whl/torch-1.10.1+cu113.html
!pip install ./tsl

In particular, the second command is installing `torch-geometric` dependencies for the specific environment you have on colab with GPU runtime. Please refer to [PyG installation guidelines](https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html) for installation in other environments.

We recommend to install tsl from GitHub repository at the moment, to be sure you are up-to-date with latest version.

Let's check if everything is ok.

In [1]:
import tsl
import torch
import numpy as np

np.set_printoptions(suppress=True)
tsl.logger.disabled = True

print(f"tsl version  : {tsl.__version__}")
print(f"torch version: {torch.__version__}")

tsl version  : 0.1.0
torch version: 1.11.0+cu102


/home/lorenzo/anaconda3/envs/GDL/lib/python3.9/site-packages/tqdm-4.64.0-py3.9.egg/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Usage
---

tsl is more than a collection of layers. We can classify the library modules into:

* __Data loading modules__
    Manage how to store, preprocess and visualize spatio-temporal data
* __Inference modules__
    Methods and models exploiting the data to make inferences

We will go deeper on them in next sections.

## Data loading
---

`tsl` comes with several datasets used in spatiotemporal processing literature. You can find them inside the submodule `tsl.datasets`.

### Loading the dataset

As an example, we start by using the [MetrLA](https://paperswithcode.com/sota/traffic-prediction-on-metr-la) dataset, a common benchmark for traffic forecasting. The dataset contains traffic readings collected from 207 loop detectors on highways in Los Angeles County, aggregated in 5 minute intervals for 4 months between March 2012 to June 2012. Loading the dataset is as simple as that:

In [2]:
from tsl.datasets import AirQuality as AQ

dataset = AQ()

/home/lorenzo/anaconda3/envs/GDL/lib/python3.9/site-packages/torch-1.11.0-py3.9-linux-x86_64.egg/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


We can see that data are organized a 3-dimensional array, with:

* 34.272 temporal steps (1 each 5 minute for 4 months)
* 207 nodes (the loop detectors)
* 1 channels (detected speed)

Nice! Other than storing the data of interest, the dataset comes with useful tools.

In [3]:
print(f"Sampling period: {dataset.freq}\n"
      f"Has missing values: {dataset.has_mask}\n"
      f"Percentage of missing values: {(1 - dataset.mask.mean()) * 100:.2f}%\n"
      f"Has dataset exogenous variables: {dataset.has_exogenous}\n"
      f"Relevant attributes: {', '.join(dataset.attributes.keys())}")

Sampling period: <Hour>
Has missing values: True
Percentage of missing values: 25.67%
Has dataset exogenous variables: False
Relevant attributes: dist


Let's look at the output. We know that the dataset has missing entries, with `dataset.mask` being a binary indicator associated with each timestep, node and channel (with ones indicating valid values).

Also, the dataset has no exogenous variables, i.e., there are no time-varying features paired with the main signal.
Instead it has a useful attribute: the distance matrix. We call *attributes*, features that are static.

You can access exogenous variables and attributes by `dataset.name`:

In [4]:
print(dataset.dist)

[[   0.           10.01553231   19.62147943 ... 1903.71716932
  1796.30609352 1795.96589939]
 [  10.01553231    0.           10.11719073 ... 1895.15226463
  1787.72195802 1787.39244677]
 [  19.62147943   10.11719073    0.         ... 1885.08052065
  1777.64679953 1777.31922415]
 ...
 [1903.71716932 1895.15226463 1885.08052065 ...    0.
   107.63326768  107.79738345]
 [1796.30609352 1787.72195802 1777.64679953 ...  107.63326768
     0.            3.6701503 ]
 [1795.96589939 1787.39244677 1777.31922415 ...  107.79738345
     3.6701503     0.        ]]


This matrix stores the pairwise distance between sensors, with `inf` denoting two non-neighboring sensors.

Let's now check how the speed readings look like.

In [5]:
dataset.dataframe().head(10)

nodes,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,311002,311003,311004,311005,371001,371002,371003,371004,372001,372002
channels,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
time,,,,,,,,,,,,,,,,,,,,,
2014-05-01 00:00:00,138.0,89.0,105.0,98.0,109.0,87.0,88.0,91.0,87.0,87.0,...,84.112572,71.215103,45.315453,39.634724,45.188107,57.007835,63.501968,61.792046,75.698006,77.257301
2014-05-01 01:00:00,124.0,85.0,121.0,107.0,101.0,99.0,105.0,102.0,103.0,94.0,...,82.447189,71.837502,45.932846,42.260418,46.544907,54.195034,57.988342,59.111382,70.341698,72.044922
2014-05-01 02:00:00,127.0,88.0,130.0,115.0,102.0,109.0,114.0,108.0,112.0,109.0,...,78.896912,72.029999,46.272568,41.574722,46.479549,54.222298,57.437904,58.276581,69.585495,71.153893
2014-05-01 03:00:00,129.0,100.0,137.0,123.0,108.0,118.0,118.0,109.0,117.0,111.0,...,78.969444,73.082085,45.956978,42.710938,46.156586,50.423290,53.261192,55.609005,65.937805,68.518791
2014-05-01 04:00:00,119.0,109.0,144.0,129.0,115.0,124.0,130.0,116.0,124.0,114.0,...,75.767670,70.022392,43.757290,38.416008,46.463467,48.486259,52.067562,53.363712,63.745544,65.624329
2014-05-01 05:00:00,127.0,120.0,142.0,138.0,123.0,134.0,140.0,127.0,132.0,121.0,...,73.315071,68.437187,45.163715,38.913403,46.257393,45.349560,48.231865,51.182884,59.617985,59.522064
2014-05-01 06:00:00,124.0,127.0,112.0,146.0,142.0,142.0,142.0,134.0,138.0,113.0,...,74.613335,69.719688,45.270348,37.547012,43.457443,44.056183,49.775440,51.103504,61.608974,62.009888
2014-05-01 07:00:00,120.0,114.0,108.0,144.0,146.0,140.0,150.0,136.0,142.0,135.0,...,73.320137,67.786980,44.284027,39.611458,42.539032,43.371082,49.522362,50.645584,59.658863,60.610733


### Get connectivity

Besides the time series, to properly use graph-based models, we need to __connect__ nodes somehow.

With the method `dataset.get_similarity()` we can retrieve nodes' similarities computed with different methods. The available similarity methods for a dataset can be found at `dataset.similarity_options`, while the default one is at `dataset.similarity_score`.

In [6]:
print(f"Default similarity: {dataset.similarity_score}\n"
      f"Available similarity options: {dataset.similarity_options}\n")

sim = dataset.get_similarity("distance")  # same as dataset.get_similarity()
print(sim[:10, :10])  # just check first 10 nodes for readability

Default similarity: distance
Available similarity options: {'distance'}

[[1.         0.86853818 0.58219195 0.26736404 0.12036526 0.72780463
  0.48905501 0.52840458 0.32986479 0.27389788]
 [0.86853818 1.         0.86604411 0.53188295 0.29106698 0.92904057
  0.68073739 0.77580368 0.60992591 0.52348866]
 [0.58219195 0.86604411 1.         0.84114444 0.58530849 0.81783541
  0.58075869 0.73745742 0.73309012 0.62165892]
 [0.26736404 0.53188295 0.84114444 1.         0.90157624 0.52029781
  0.36513726 0.51747668 0.66768938 0.56361474]
 [0.12036526 0.29106698 0.58530849 0.90157624 1.         0.27983269
  0.18380225 0.2894282  0.45086383 0.37104449]
 [0.72780463 0.92904057 0.81783541 0.52029781 0.27983269 1.
  0.88357588 0.9437749  0.77927832 0.7185383 ]
 [0.48905501 0.68073739 0.58075869 0.36513726 0.18380225 0.88357588
  1.         0.96315672 0.81181861 0.82296991]
 [0.52840458 0.77580368 0.73745742 0.51747668 0.2894282  0.9437749
  0.96315672 1.         0.90479142 0.88158209]
 [0.32986479 0.6

With this method, we compute weight $w_t^{i,j}$ of the edge connecting $i$-th and $j$-th node as
$$
w^{i,j} = \left\{\begin{array}{cl}
     \exp \left(-\frac{\operatorname{dist}\left(i, j\right)^{2}}{\gamma}\right) & \operatorname{dist}\left(i, j\right) \leq \delta  \\
     0 & \text{otherwise}
\end{array}\right. ,
$$
where $\operatorname{dist}\left(i, j\right)$ is the distance between $i$-th and $j$-th node, $\gamma$ controls the width of the kernel and $\delta$ is a threshold. Notice that in this case the similarity matrix is not symmetric, since the original preprocessed distance matrix is not symmetric too.

So far so good, now we can build an adjacency matrix out ouf the computed similarity.

The method `dataset.get_connectivity()`, wraps the `dataset.get_similarity()` module, provides useful preprocessing options, and, eventually, returns a sparse (weighted) adjacency matrix.

In [7]:
adj = dataset.get_connectivity(threshold=0.1,
                               include_self=False,
                               normalize_axis=1,
                               layout="edge_index")

With this function call, under the hood we:

1. compute the similarity matrix as before
1. set to $0$ values below $0.1$ (`threshold=0.1`)
1. remove self loops (`include_self=False`)
1. normalize edge weights by the in degree of nodes (`normalize_axis=1`)
1. request the sparse COO layout of PyG (`layout="edge_index"`)

The connectivity matrix with `edge_index` layout is provided in COO format, adopting the convention and notation used in PyTorch Geometric. The returned connectivity is a tuple (`edge_index`, `edge_weight`), where `edge_index` lists all edges as pairs of source-target nodes (dimensions `[2, E]`) and `edge_weight` (dimension `[E]`) stores the corresponding weights.

In [8]:
edge_index, edge_weight = adj

print(edge_index.shape)
print(edge_weight)

(2, 5398)
[0.06717467 0.04276256 0.0225386  ... 0.34616419 0.99999995 0.99999995]


The dense layout corresponds to the weighted adjacency matrix $A \in \mathbb{R}^{N \times N}$. The module `tsl.ops.connectivity` contains useful operations for connectivities, including methods to change layout.

In [9]:
from tsl.ops.connectivity import edge_index_to_adj

dense = edge_index_to_adj(edge_index, edge_weight)
print(dense.shape)

(437, 437)


## Data processing
---

In this section, we will see how to transfer data from a dataset to an inference model (e.g., a spatiotemporal graph neural network).

### The SpatioTemporalDataset

The first class that comes in help is `tsl.data.SpatioTemporalDataset`. This class is a subclass of `torch.utils.data.Dataset` can be considered as wrapper of a `tsl` dataset providing the interface for further processing.

In particular, a `SpatioTemporalDataset` object can be used to achieve the following:
* perform the transformations required to feed the data to a model (e.g., casting to `torch.tensor`, handling different `shapes`)
* handling temporal slicing and windowing for training (e.g., split data in $\left( \text{window}, \text{horizon} \right)$ samples)
* defining the layout of inputs and targets (e.g., how node attributes and exogenous variables are arranged)
* preprocess data before creating a batch

Let's see how to go from a `Dataset` to a `SpatioTemporalDataset`.

In [10]:
from tsl.data import SpatioTemporalDataset

torch_dataset = SpatioTemporalDataset(*dataset.numpy(return_idx=True),
                                      connectivity=adj,
                                      mask=dataset.mask,
                                      horizon=12,
                                      window=12)
print(torch_dataset)

SpatioTemporalDataset(n_samples=8737, n_nodes=437, n_channels=1)


As you can see, the number of samples is not the same as the number of steps we have in the dataset. Indeed, we divided the time series with a sliding window of 12 steps for the input (`window=12`), with a corresponding horizon of 12 steps (`horizon=12`). Thus, a sample spans for a total of $24$ steps. But let's look in details to the layout of a sample:

In [11]:
sample = torch_dataset[0]
print(sample)

Data(input:{x=[12, 437, 1], edge_index=[2, 5398], edge_weight=[5398]}, target:{y=[12, 437, 1]}, has_mask=True)


A sample has 5 main attributes:

* `sample.input` is a mapping of data to be forwarded as input to the model.
* `sample.target` is a mapping of data to be forwarded as target for the loss function of the model.
* `sample.mask` store the `mask`, if any. It is useful for computing the loss only on valid data.
* `sample.transform` is a mapping containing as value a transformation function (e.g., scaling, detrending) and as key the name of the tensor to be transformed.
* `sample.pattern` stores the pattern, i.e., a more informative shape representation, of each tensor in `sample`.

Let's check more in details how each of these attributes is composed.

#### Input and Target

A sample is a `tsl.data.Data` object which stores all that is needed to support inference.
Both `input` and `target` are `tsl.data.DataView` of this storage.
This means that they have the same methods, but contain different subsets keys.
As a results, you cannot store two tensors using the key in `input` and `target`.

In [12]:
sample.input.to_dict()

{'x': tensor([[[138.0000],
          [ 89.0000],
          [105.0000],
          ...,
          [ 61.7920],
          [ 75.6980],
          [ 77.2573]],
 
         [[124.0000],
          [ 85.0000],
          [121.0000],
          ...,
          [ 59.1114],
          [ 70.3417],
          [ 72.0449]],
 
         [[127.0000],
          [ 88.0000],
          [130.0000],
          ...,
          [ 58.2766],
          [ 69.5855],
          [ 71.1539]],
 
         ...,
 
         [[147.0000],
          [133.0000],
          [148.0000],
          ...,
          [ 52.9467],
          [ 64.2753],
          [ 63.6968]],
 
         [[188.0000],
          [ 29.0000],
          [167.0000],
          ...,
          [ 53.2139],
          [ 63.2178],
          [ 63.1341]],
 
         [[212.0000],
          [ 33.3333],
          [178.0000],
          ...,
          [ 55.3667],
          [ 64.9315],
          [ 65.9394]]]),
 'edge_index': tensor([[  0,   0,   0,  ..., 434, 435, 436],
         [  1,   2

In [13]:
sample.target.to_dict()

{'y': tensor([[[229.0000],
          [111.0000],
          [179.0000],
          ...,
          [ 55.8811],
          [ 68.1196],
          [ 66.7615]],
 
         [[240.0000],
          [172.0000],
          [184.0000],
          ...,
          [ 53.8635],
          [ 67.5794],
          [ 67.3180]],
 
         [[240.0000],
          [173.0000],
          [188.0000],
          ...,
          [ 52.7024],
          [ 63.6870],
          [ 63.7933]],
 
         ...,
 
         [[119.0000],
          [ 51.0000],
          [ 50.0000],
          ...,
          [ 61.6110],
          [ 73.4010],
          [ 70.2052]],
 
         [[ 48.0000],
          [ 33.0000],
          [ 36.0000],
          ...,
          [ 67.3310],
          [ 78.3555],
          [ 77.4719]],
 
         [[ 37.0000],
          [ 31.0000],
          [ 34.0000],
          ...,
          [ 63.4985],
          [ 76.7035],
          [ 82.3329]]])}

#### Mask and Transform

`mask` and `transform` are just symbolic links to the corresponding object inside the storage. They also expose properties `has_mask` and `has_transform`.

In [14]:
if sample.has_mask:
    print(sample.mask)
else:
    print("Sample has no mask.")

tensor([[[1],
         [1],
         [1],
         ...,
         [0],
         [0],
         [0]],

        [[1],
         [1],
         [1],
         ...,
         [0],
         [0],
         [0]],

        [[1],
         [1],
         [1],
         ...,
         [0],
         [0],
         [0]],

        ...,

        [[1],
         [1],
         [1],
         ...,
         [0],
         [0],
         [0]],

        [[1],
         [1],
         [1],
         ...,
         [0],
         [0],
         [0]],

        [[1],
         [1],
         [1],
         ...,
         [0],
         [0],
         [0]]], dtype=torch.uint8)


In [15]:
if sample.has_transform:
    print(sample.transform)
else:
    print("Sample has no transform functions.")

Sample has no transform functions.


#### Pattern

The `pattern` mapping can be useful to glimpse on how data are arranged.
The convention we use is the following:

* "b" stands for "batch size"
* "c" stands for "number of channels" (per node)
* "e" stands for "number edges"
* "n" stands for "number of nodes"
* "s" stands for "number of time steps"


In [16]:
sample.pattern

{'x': 's n c',
 'edge_index': '2 e',
 'edge_weight': 'e',
 'mask': 's n c',
 'y': 's n c'}

### The SpatioTemporalDataModule

Usually, before running an experiment there are two quite common preprocessing steps:

* splitting the dataset into training/validation/test sets
* data preprocessing (scaling/normalizing data, detrending)

In `tsl`, these operations are carried out in the `tsl.data.SpatioTemporalDataModule`, which is based on `pytorch-lightning`'s data modules.

Let's see an example

In [17]:
from tsl.data import SpatioTemporalDataModule
from tsl.data.preprocessing import StandardScaler

scalers = {'data': StandardScaler(axis=(0, 1))}

splitter = dataset.get_splitter(val_len=0.1, test_len=0.2)

dm = SpatioTemporalDataModule(
    dataset=torch_dataset,
    scalers=scalers,
    splitter=splitter,
    batch_size=64,
)

print(dm)

SpatioTemporalDataModule(train_len=None, val_len=None, test_len=None, scalers=[data], batch_size=64)


 Eventually one could extend the base datamodule to add further processing in case it is needed.

At this point, the `DataModule` object has not actually performed any processing yet (lazy approach).

We can execute the preprocessing routines by calling `setup` method.

Note that

In [18]:
dm.setup()
print(dm)

SpatioTemporalDataModule(train_len=5141, val_len=576, test_len=2884, scalers=[data], batch_size=64)


After `setup` has been called, the datamodule carries the following operations:

1. Carries out the dataset splitting into training/validation/test sets according to the `splitter` argument.
1. Fits all the `Scalers` on the training data in `dataset` corresponding to the scalers' keys.

#### Scalers

The `tsl.data.preprocessing` package offers several of the most common data normalization techniques under the `tsl.data.preprocessing.Scaler` interface.
They adopt an API similar to `scikit-learn`'s scalers, with `fit`/`transform`/`fit_transform`/`inverse_transform` methods. Check the documentation for more details about this.


## Building a Model
---

In this section, we will see how to build a very simple Graph Neural Network for Spatiotemporal data.
All the neural network code inside `tsl` is under the `tsl.nn` module.


### The NN module

The `tsl.nn` module is organized as follows:

```
tsl
└── nn
    ├── base
    ├── blocks
    ├── layers
    ├── models
    ├── metrics
    ├── ops
    └── utils
```

The 3 most important submodules in it are `layers`, `blocks`, and `models`, ordered by increasing level of abstraction.

#### Layers

A _layer_ is a basic building block for our neural networks. In simple words, a layer takes an input, performs one (or few) operations, and return a transformation of the input. Examples of layers are `DiffConv`, which implements [diffusion convolution](https://arxiv.org/abs/1707.01926), or `LayerNorm`.

#### Blocks

_blocks_ perform more complex transformations or combine several operations. We divide blocks into _encoders_, if they provide a representation of the input in a new space, and _decoders_, if they produce a meaningful output from a representation.

#### Models

We wrap a series of operations, represented by blocks and/or layers, in a _model_. A model is meant to takes as input a batch `SpatioTemporalDataset`'s items and return the desired output.

Let's create a very simple model with a RNN encoder and a nonlinear GCN readout.
To do so, we import `RNN` from the encoders and `GCNDecoder` from the decoders in the `tsl.nn.blocks` module.

In [19]:
from tsl.nn.blocks.encoders import RNN
from tsl.nn.blocks.decoders import GCNDecoder


class TimeThenSpaceModel(torch.nn.Module):
    def __init__(self,
                 input_size,
                 hidden_size,
                 rnn_layers,
                 gcn_layers,
                 horizon):
        super(TimeThenSpaceModel, self).__init__()

        self.input_encoder = torch.nn.Linear(input_size, hidden_size)

        self.encoder = RNN(input_size=hidden_size,
                           hidden_size=hidden_size,
                           n_layers=rnn_layers)

        self.decoder = GCNDecoder(
            input_size=hidden_size,
            hidden_size=hidden_size,
            output_size=input_size,
            horizon=horizon,
            n_layers=gcn_layers
        )

    def forward(self, x, edge_index, edge_weight):
        # x: [batches steps nodes channels]
        x = self.input_encoder(x)

        x = self.encoder(x, return_last_state=True)

        return self.decoder(x, edge_index, edge_weight)

Fine, we have a model and we have data, now let's train it!

## Setting up training
---

### The Predictor

In `tsl`, inference engines are implemented as a [`LightningModule`](https://pytorch-lightning.readthedocs.io/en/latest/api/pytorch_lightning.core.LightningModule.html#pytorch_lightning.core.LightningModule). `tsl.predictors.Predictor` is a base class that can be extended to build more complex forecasting approaches.
These modules are meant to wrap deep models in order to ease training and inference phases.

In [20]:
from tsl.nn.metrics.metrics import MaskedMAE, MaskedMAPE
from tsl.predictors import Predictor

loss_fn = MaskedMAE(compute_on_step=True)

metrics = {'mae': MaskedMAE(compute_on_step=False),
           'mape': MaskedMAPE(compute_on_step=False),
           'mae_at_15': MaskedMAE(compute_on_step=False, at=2),  # `2` indicated the third time step,
                                                                 # which correspond to 15 minutes ahead
           'mae_at_30': MaskedMAE(compute_on_step=False, at=5),
           'mae_at_60': MaskedMAE(compute_on_step=False, at=11), }

model_kwargs = {
    'input_size': dm.n_channels,  # 1 channel
    'horizon': dm.horizon,  # 12, the number of steps ahead to forecast
    'hidden_size': 32,
    'rnn_layers': 1,
    'gcn_layers': 2
}

# setup predictor
predictor = Predictor(
    model_class=TimeThenSpaceModel,
    model_kwargs=model_kwargs,
    optim_class=torch.optim.Adam,
    optim_kwargs={'lr': 0.001},
    loss_fn=loss_fn,
    metrics=metrics
)

/home/lorenzo/anaconda3/envs/GDL/lib/python3.9/site-packages/pytorch_lightning-1.6.0-py3.9.egg/pytorch_lightning/utilities/parsing.py:244: UserWarning: Attribute 'loss_fn' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss_fn'])`.
  rank_zero_warn(


Now let's finalize the last details. We make use of [TensorBoard](https://www.tensorflow.org/tensorboard/) to log and visualize metrics.

In [21]:
# from pytorch_lightning.loggers import TensorBoardLogger


# logger = TensorBoardLogger(save_dir="logs", name="tsl_intro", version=0)

In [22]:
from pytorch_lightning.loggers import CSVLogger

logger = CSVLogger(save_dir='prova' , name='savedata')

In [ ]:

#%load_ext tensorboard
#%tensorboard --logdir logs

We let `pytorch_lightning.Trainer` handle the dirty work for us. We can directly pass the datamodule to the trainer for fitting.

If this is the case, the trainer will call the `setup` method, and then load train and validation sets.

In [24]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    dirpath='logs',
    save_top_k=1,
    monitor='val_mae',
    mode='min',
)

trainer = pl.Trainer(max_epochs=3,
                     logger=logger,
                     gpus=1 if torch.cuda.is_available() else None,
                     limit_train_batches=100,
                     callbacks=[checkpoint_callback])

trainer.fit(predictor, datamodule=dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/lorenzo/anaconda3/envs/GDL/lib/python3.9/site-packages/pytorch_lightning-1.6.0-py3.9.egg/pytorch_lightning/callbacks/model_checkpoint.py:608: UserWarning: Checkpoint directory /home/lorenzo/PycharmProjects/gdl_air_quality/code/utility notebooks/logs exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name          | Type               | Params
-----------------------------------------------------
0 | loss_fn       | MaskedMAE          | 0     
1 | train_metrics | MetricCollection   | 0     
2 | val_metrics   | MetricCollection   | 0     
3 | test_metrics  | MetricCollection   | 0     
4 | model         | TimeThenSpaceModel | 12.0 K
-----------------------------------------------------
12.0 K    Trainable params
0         Non-trainable params
12.0 K    Total params
0.048     Total e

Sanity Checking: 0it [00:00, ?it/s]

/home/lorenzo/anaconda3/envs/GDL/lib/python3.9/site-packages/pytorch_lightning-1.6.0-py3.9.egg/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/home/lorenzo/anaconda3/envs/GDL/lib/python3.9/site-packages/pytorch_lightning-1.6.0-py3.9.egg/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 2: 100%|██████████| 89/89 [10:27<00:00,  7.05s/it, loss=26.5, v_num=1, val_mae=25.10, val_mae_at_15=18.90, val_mae_at_30=25.80, val_mae_at_60=33.30, val_mape=0.620, train_mae=26.30, train_mae_at_15=20.60, train_mae_at_30=27.30, train_mae_at_60=33.40, train_mape=0.616]   


## Testing
---


Now let's see how the trained model behaves on new unseen data.

In [25]:
predictor.load_model(checkpoint_callback.best_model_path)
predictor.freeze()

performance = trainer.test(predictor, datamodule=dm)

/home/lorenzo/anaconda3/envs/GDL/lib/python3.9/site-packages/pytorch_lightning-1.6.0-py3.9.egg/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing DataLoader 0: 100%|██████████| 46/46 [00:46<00:00,  1.01s/it]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           24.076353073120117
        test_mae            24.635517120361328
     test_mae_at_15         19.228456497192383
     test_mae_at_30         25.976608276367188
     test_mae_at_60          31.25102424621582
        test_mape           0.6429881453514099
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


Cool! We succeeded in creating first a simple, yet effective, SpatioTemporal model!

We are now __tsl ninjas__. We learned how to:

* Load benchmark datasets
* Organize data for processing
* Preprocess the data
* Build a Spatiotemporal GNN
* Train and evaluate the model

We hope you enjoyed this introduction to `tsl`, do not hesitate to contact us if you have any question or problem while using it.

The tsl team.

🧡